# <center>Лабораторная работа №16
## <center>Предсказывание просмотров wiki-страниц

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import numpy np


from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests

print(__version__)

init_notebook_mode(connected = True)

In [ ]:
def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

**Заполните пропущенный код.**

## Подготавливаем данные

In [ ]:
df = pd.read_csv('wiki_machine_learning.csv', sep = ' ')
df = df[df['count'] != 0]
df.head()

In [ ]:
df.shape

## Предсказание с помощью Facebook Prophet

Для начала построим предсказание с помощью простой библиотеки `Facebook Prophet`. Для того, чтобы посмотреть на качество модели, отбросим из обучающей выборки последние 30 дней.

In [ ]:
df.date = pd.to_datetime(df.date)

In [ ]:
plotly_df(df.set_index('date')[['count']])

In [ ]:
from fbprophet import Prophet

In [ ]:
predictions = 30

df = df[['date', 'count']]
df.columns = ['ds', 'y']
df.tail()

In [ ]:
train_df = #Ваш код здесь

In [ ]:
m = Prophet()
m.fit(train_df);

In [ ]:
future = #Ваш код здесь
future.tail()

In [ ]:
forecast = m.predict(future)
forecast.tail()

<span style="color:red; font-size:2em;">Задание 1</span>

**Какое предсказание числа просмотров wiki-страницы на 20 января? Ответ округлите до целого числа.**

Оценим качество предсказания по последним 30 точкам.

In [ ]:
m.plot(forecast)

In [ ]:
m.plot_components(forecast)

In [ ]:
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 
                                   'yhat_upper']].join(df.set_index('ds'))

In [ ]:
import numpy as np
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100*cmp_df['e']/cmp_df['y']

#Ваш код здесь

print('MAPE')
print('MAE')

Оценим качество предсказания по последним 30 точкам.

<span style="color:red; font-size:2em;">Задание 2</span>

**Выведете значение MAPE**

<span style="color:red; font-size:2em;">Задание 3</span>

**Выведете значение MAE**

## Предсказываем с помощью ARIMA

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
plt.rcParams['figure.figsize'] = (15, 10)

<span style="color:red; font-size:2em;">Задание 4</span>

**Проверим стационарность ряда с помощью критерия Дики-Фулера. Является ли ряд стационарным? Какое значение p-value?** 

In [ ]:
# Ваш код здесь
print("Критерий Дики-Фуллера:")

In [ ]:
# Пример построения графиков временных рядов
ax = plt.subplot(211)
sm.graphics.tsa.plot_acf(train_df.y_diff[13:].values.squeeze(), lags=48, ax=ax)

ax = plt.subplot(212)
sm.graphics.tsa.plot_pacf(train_df.y_diff[13:].values.squeeze(), lags=48, ax=ax)

Начальные приближения:
* Q = 1
* q = 3
* P = 3
* p = 1

In [ ]:
ps = range(0, 2)
ds = range(0, 2)
qs = range(0, 4)
Ps = range(0, 4)
Ds = range(0, 3)
Qs = range(0, 2)

In [ ]:
from itertools import product

parameters = product(ps, ds, qs, Ps, Ds, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
%%time
import warnings
from tqdm import tqdm
results1 = []
best_aic = float("inf")
warnings.filterwarnings('ignore')

for param in tqdm(parameters_list):
    #try except нужен, потому что на некоторых наборах параметров модель не обучается
    try:
        model=sm.tsa.statespace.SARIMAX(train_df['y'], order=(param[0], param[1], param[2]), 
                                        seasonal_order=(param[3], param[4], param[5], 7)).fit(disp=-1)
    #выводим параметры, на которых модель не обучается и переходим к следующему набору
    except (ValueError, np.linalg.LinAlgError):
        continue
    aic = model.aic
    #сохраняем лучшую модель, aic, параметры
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results1.append([param, model.aic])

In [ ]:
result_table1 = pd.DataFrame(results1)
result_table1.columns = ['parameters', 'aic']
print(result_table1.sort_values(by = 'aic', ascending=True).head())

Если рассматривать предложенные в форме варианты:

In [ ]:
result_table1[result_table1['parameters'].isin([(1, 0, 2, 3, 1, 0),
                                                (1, 1, 2, 3, 2, 1),
                                                (1, 1, 2, 3, 1, 1),
                                                (1, 0, 2, 3, 0, 0)])]

Теперь то же самое, но для ряда с преобразованием Бокса-Кокса.

In [ ]:
import scipy.stats
# Выведите оптимальный параметр преобразования Бокса-Кокса

# Ваш код здесь
print("Оптимальный параметр преобразования Бокса-Кокса:")

In [ ]:
results2 = []
best_aic = float("inf")

for param in tqdm(parameters_list):
    #try except нужен, потому что на некоторых наборах параметров модель не обучается
    try:
        model=sm.tsa.statespace.SARIMAX(train_df['y_box'], order=(param[0], param[1], param[2]), 
                                        seasonal_order=(param[3], param[4], param[5], 7)).fit(disp=-1)
    #выводим параметры, на которых модель не обучается и переходим к следующему набору
    except (ValueError, np.linalg.LinAlgError):
        continue
    aic = model.aic
    #сохраняем лучшую модель, aic, параметры
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results2.append([param, model.aic])
    
warnings.filterwarnings('default')

In [ ]:
result_table2 = pd.DataFrame(results2)
result_table2.columns = ['parameters', 'aic']
print(result_table2.sort_values(by = 'aic', ascending=True).head())

Если рассматривать предложенные в форме варианты:

In [ ]:
result_table2[result_table2['parameters'].isin([(1, 0, 2, 3, 1, 0),
                                                (1, 1, 2, 3, 2, 1),
                                                (1, 1, 2, 3, 1, 1),
                                                (1, 0, 2, 3, 0, 0)])].sort_values(by = 'aic')

<span style="color:red; font-size:2em;">Задание 5</span>

**Далее перейдем к построению модели SARIMAX (`sm.tsa.statespace.SARIMAX`). Модель c какими параметрами лучшая по `AIC`-критерию?** 

Посмотрим на прогноз лучшей по AIC модели.

In [ ]:
# Ваш код здесь для построя AIC модели